# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

The environment is already saved in the Workspace and can be accessed at the file path provided below. 

In [2]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
for i in range(5):                                         # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.04500000085681677
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291
Total score (averaged over agents) this episode: -0.004999999888241291


When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [6]:
from collections import deque
from ddpg_agent import Agent
import torch
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
def ddpg(n_episodes=10000, max_t=1000, num_agents=20, rolling_window=100):
    """"
    Deep Deterministic Policy Gradient (DDPG)
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    """
    
    score_rolling_window = deque(maxlen=rolling_window)
    score_list = []
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]           # reset the environment
        state = env_info.vector_observations                        # get current state
        for agent in agents:
            agent.reset()                                           # reset agents
        score = np.zeros(num_agents)
        
        time_start = time.time()
        for step in range(max_t):
            agents_actions = []
            for a, agent in enumerate(agents):
                agents_actions.append(agent.act(state[a]))          # select an action from every agent
                                              
            env_info = env.step(agents_actions)[brain_name]         # perform action within the environment
            next_states = env_info.vector_observations              # get next state after action
            rewards = env_info.rewards                              # receive reward
            done = env_info.local_done                              # check if episode is done
            
            for a, agent in enumerate(agents):                      # perform agents step
                agent.step(state[a], 
                           agents_actions[a], 
                           rewards[a], 
                           next_states[a], 
                           done[a])
            
            state = next_states                                     # update state
            score += rewards                                        # add rewards
            
            if np.any(done):
                break 
                
        avg_score = np.mean(score)
        score_list.append(avg_score)

        score_rolling_window.append(avg_score)
        avg_rolling_score = np.mean(score_rolling_window)
        
        
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.3f}\tTime: {:.2f}'.format(i_episode, 
                                                                                        avg_rolling_score, 
                                                                                        avg_score, 
                                                                                        time.time() - time_start), end="")
        torch.save(agent.actor_local.state_dict(), 'checkpoint_ddpg_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_ddpg_critic.pth')
        
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, avg_rolling_score))
        if avg_score >= 0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, avg_rolling_score))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_ddpg_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_ddpg_critic.pth')
            break
            
    return score_list

In [ ]:
agents = []

for a in range(num_agents): 
    agents.append(Agent(state_size=state_size, action_size=action_size, random_seed=23))

score_list = ddpg(num_agents=2)

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(score_list) + 1), score_list)
plt.ylabel('Score')
plt.xlabel('Episodes')
plt.show()

Episode 100	Average Score: -0.00	Score: -0.005	Time: 2.22
Episode 200	Average Score: -0.00	Score: -0.005	Time: 2.44
Episode 300	Average Score: -0.00	Score: -0.005	Time: 2.52
Episode 400	Average Score: -0.00	Score: -0.005	Time: 2.41
Episode 500	Average Score: -0.00	Score: -0.005	Time: 2.41
Episode 600	Average Score: -0.00	Score: -0.005	Time: 2.49
Episode 700	Average Score: -0.00	Score: -0.005	Time: 2.52
Episode 800	Average Score: -0.00	Score: -0.005	Time: 2.39
Episode 900	Average Score: -0.00	Score: -0.005	Time: 2.43
Episode 1000	Average Score: 0.00	Score: -0.005	Time: 2.27
Episode 1100	Average Score: 0.01	Score: -0.005	Time: 2.152
Episode 1200	Average Score: 0.00	Score: -0.005	Time: 2.21
Episode 1300	Average Score: -0.00	Score: -0.005	Time: 2.27
Episode 1400	Average Score: 0.01	Score: -0.005	Time: 2.232
Episode 1500	Average Score: 0.01	Score: 0.045	Time: 5.310
Episode 1600	Average Score: 0.02	Score: 0.045	Time: 4.962
Episode 1700	Average Score: 0.01	Score: -0.005	Time: 2.28
Episode 180